# Finetuning LLaMa + Text-to-SQL 

This walkthrough shows you how to fine-tune LLaMa-7B on a Text-to-SQL dataset, and then use it for inference against
any database of structured data using LlamaIndex.

**NOTE**: This code is taken and adapted from Modal's `doppel-bot` repo: https://github.com/modal-labs/doppel-bot.
**NOTE**: A lot of the code is contained in the underlying Python scripts in the `src` directory. We definitely encourage you to go and take a look!

### Setup

NOTE: you will need to setup a Modal account + token in order to use this notebook.

In [2]:
%pip install -r requirements.txt

  Obtaining dependency information for modal-client==0.50.3044 from https://files.pythonhosted.org/packages/23/9a/5359c8262c7af5b53d971d912bbacbe81fb5e1824969fb061137926608c9/modal_client-0.50.3044-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index==0.8.2.post1 from https://files.pythonhosted.org/packages/91/ee/590f6f6d5ba81eb8c08c9a98dad290f983d0f609e51723821b3684f468a1/llama_index-0.8.2.post1-py3-none-any.whl.metadata
  Obtaining dependency information for datasets==2.14.4 from https://files.pythonhosted.org/packages/66/f8/38298237d18d4b6a8ee5dfe390e97bed5adb8e01ec6f9680c0ddf3066728/datasets-2.14.4-py3-none-any.whl.metadata
  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/37/1a/8d20e8704da9fa070eb909265584b960da57be1d833d550c59f50906dc5c/peft-0.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for asgiref from https://files.pythonhosted.org/packages/9b/80/b9051a4a07ad231558fcd8ffc89232711b4e618c15cb7a392

### Load Training Data for Finetuning LLaMa

We load data from `b-mc2/sql-create-context` on Hugging Face: https://huggingface.co/datasets/b-mc2/sql-create-context.

This dataset consists of tuples of natural language queries, create table statements, and ground-truth SQL queries. This is the dataset that we use to finetune our SQL model.

In [4]:
data_dir = "data_sql"

!modal run src.load_data_sql --data-dir {data_dir}

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Tmax\AppData\Local\Programs\Python\Python311\Scripts\modal.exe\__main__.py", line 7, in <module>
  File "c:\Users\Tmax\AppData\Local\Programs\Python\Python311\Lib\site-packages\modal\__main__.py", line 6, in main
    entrypoint_cli()
  File "c:\Users\Tmax\AppData\Local\Programs\Python\Python311\Lib\site-packages\click\core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Tmax\AppData\Local\Programs\Python\Python311\Lib\site-packages\typer\core.py", line 778, in main
    return _main(
           ^^^^^^
  File "c:\Users\Tmax\AppData\Local\Programs\Python\Python311\Lib\site-packages\typer\core.py", line 216, in _main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "c:\Users\Tmax\AppData\Local\Programs\Python\Python311\Lib\site-packages\click\core.py", l

### Run Finetuning Script

We run our finetuning script on the loaded dataset.
The finetuning script contains the following components:
- We split the dataset into training and validation splits.
- We format each split into input/output tuples of token id's. This means that the labels are the same as inputs (loss signal is measured on full input, not just on the generated portion). 
- We use `LoraConfig` from `peft` for efficient fine-tuning.
- We use `transformers.Trainer` to actually run the training process.
- If a valid `WANDB_PROJECT` is specified, along with the relevant secret in Modal, then we will log results to wandb.

We use Modal to spin up an A100 to run our finetuning code. 

In [ ]:
data_dir = "data_sql"

!modal run src.finetune_sql --data-dir {data_dir}

### Evaluation

We provide a basic evaluation script over sample data from `sql-create-context` so that you can see for yourself how well the finetuned model performs vs. the baseline model.

In [17]:
!modal run src.eval_sql::main

✓ Initialized. View app at https://modal.com/apps/ap-JS10MTVAhat3jK4K8QwKyv
⠋ Initializing...
⠼ Creating objects...objects...
├── ⠋ Creating download_models...
└── ⠋ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠧ Creating objects...
├── ⠸ Creating download_models...
└── ⠸ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠋ Creating objects...
├── ⠦ Creating download_models...
└── ⠦ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠸ Creating objects...
├── ⠏ Creating download_models...
└── ⠏ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠦ Creating objects...
├── ⠹ Creating download_models...
⠋ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── ⠦ Creating download_models...
⠸ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── ⠏ Creating download_models...
⠦ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── ⠹ Creating download_models...
⠏ Creating object

### Integrate Model with LlamaIndex

Now that the model is finetuned, the checkpoints and model binary are stored in a model directory (by default it is in `/vol/data_sql`).

We can now use this model in LlamaIndex for text-to-SQL applications.

Specifically, we provide an interface allowing users to define any `sqlite` data file, and then they can run queries over this data file. We first create and dump a sample `cities.db` file containing (city, population, country) tuples. We then run inference over this file.

#### Create sample db

In [9]:
# create sample 
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

In [10]:
db_file = "cities.db"
engine = create_engine(f"sqlite:///{db_file}")
metadata_obj = MetaData()

In [11]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [12]:
# insert sample rows
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Chicago", "population": 2679000, "country": "United States"},
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

#### Run Inference

In [15]:
query = "Which city has the highest population?"

!modal run src.inference_sql_llamaindex::main --query '{query}' --sqlite-file-path {db_file} --model-dir "data_sql" --use-finetuned-model True

✓ Initialized. View app at https://modal.com/apps/ap-BjYd5um2Efz70MftYxlyzE
⠋ Initializing...
⠹ Creating objects...objects...
├── ⠋ Creating download_models...
└── ⠋ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠴ Creating objects...
├── ⠸ Creating download_models...
└── ⠸ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠏ Creating objects...
├── ⠦ Creating download_models...
⠹ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── 🔨 Created download_models.
⠴ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── 🔨 Created download_models.
⠇ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── 🔨 Created download_models.
├── 🔨 Created mount /Users/jerryliu/Programming/modal_finetune_sql/src
├── ⠋ Creating run_query...
└── ⠋ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠙ Creating objects...
├── 🔨 Created download_models.
├── 🔨 Created mount /Users/jerryliu/Programming/modal_finetu

In [16]:
# you can also choose to run the original (nonfinetuned model) to compare results
# note: it throws an error 

# use non-finetuned model
!modal run src.inference_sql_llamaindex::main --query '{query}' --sqlite-file-path {db_file} --model-dir "data_sql" --use-finetuned-model False

✓ Initialized. View app at https://modal.com/apps/ap-1n1QmIubsUDYdMnVNCTGzh
⠋ Initializing...
⠹ Creating objects...objects...
├── ⠋ Creating download_models...
└── ⠋ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠴ Creating objects...
├── ⠸ Creating download_models...
└── ⠸ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠏ Creating objects...
├── 🔨 Created download_models.
⠹ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── 🔨 Created download_models.
⠴ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── 🔨 Created download_models.
⠇ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── 🔨 Created download_models.
├── 🔨 Created mount /Users/jerryliu/Programming/modal_finetune_sql/src
├── ⠋ Creating run_query...
└── ⠋ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠙ Creating objects...
├── 🔨 Created download_models.
├── 🔨 Created mount /Users/jerryliu/Programming/modal_finetune_

### (Optional) Download Model

If you want to download the model weights for your own use, just run the following script.

In [ ]:
from src.download_weights import main

main("out_model", model_dir="data_sql")